In [1]:
import pandas as pd

# Load the necessary datasets
precip_df = pd.read_excel('Daily_Colby_Mesonet.xlsx', usecols=['Timestamp', 'Precipitation'])
texture_df = pd.read_excel('24 KSU TAPS Soil texture.xlsx', skiprows=1)
ET_mesonet = pd.read_excel('ET_Grass_Mesonet.xlsx')
ET_mesonet = ET_mesonet.drop([0, 1], axis=0)
ET_mesonet = ET_mesonet.rename(columns={"ETo": "ETo Grass(mm)"})
ET_mesonet = ET_mesonet.rename(columns={"ETo.1": "ETo Alfalfa(mm)"})
ET_mesonet = ET_mesonet.rename(columns={"Precipitation": "Precipitation"})


Team_Plot_ID = pd.read_excel('team_plot_data.xlsx')


In [ ]:
import pandas as pd

# Define the path to the Excel file
file_path_sentek = '/Users/yaniksitta/albirro_project/24 KSU TAPS Sentek.xlsx'

# Load the Excel file
excel_file_sentek = pd.ExcelFile(file_path_sentek)

# Initialize an empty list to hold the data from each worksheet
all_sheets_sentek = []

# Iterate over each sheet in the Excel file
for sheet_name_sentek in excel_file_sentek.sheet_names:
    # Read the sheet, skipping the first two rows
    sheet_data_sentek = pd.read_excel(file_path_sentek, sheet_name=sheet_name_sentek, skiprows=2)
    
    # Remove the " C" from the column names if present
    sheet_data_sentek.columns = [col.replace(" C", "")
                                     .replace("Temp Sensor #6 (55 cm)", "Temp. Sensor #6 (55 cm)")
                                 for col in sheet_data_sentek.columns]
    
    # Add a new column with the sheet name (Team Number)
    sheet_data_sentek['Team Number'] = sheet_name_sentek
    
    # Append the sheet data to the list
    all_sheets_sentek.append(sheet_data_sentek)

# Combine all sheets into a single DataFrame
combined_sentek = pd.concat(all_sheets_sentek, ignore_index=True)

# Remove all rows where column V1 value is -1
combined_sentek = combined_sentek[combined_sentek["V1"] != -1]


In [ ]:
filtered_combined_sentek = combined_sentek.loc[:, ["Timestamp", "Sensor #1 (5 cm)", "Sensor #2 (15 cm)", "Sensor #3 (25 cm)", "Team Number"]]

# Round 'Timestamp' to the nearest hour
filtered_combined_sentek['Timestamp'] = pd.to_datetime(filtered_combined_sentek['Timestamp'])
filtered_combined_sentek['Rounded Timestamp'] = filtered_combined_sentek['Timestamp'].dt.round('H')

# Group by 'Rounded Timestamp' and 'Team Number', then calculate the mean for sensor columns
grouped_filtered_combined_sentek = filtered_combined_sentek.groupby(['Rounded Timestamp', 'Team Number']).agg({
    'Sensor #1 (5 cm)': 'mean',
    'Sensor #2 (15 cm)': 'mean',
    'Sensor #3 (25 cm)': 'mean'
}).reset_index()

# Calculate the weighted volumetric water content (VWC)
grouped_filtered_combined_sentek['Weighted VWC'] = (
    grouped_filtered_combined_sentek['Sensor #1 (5 cm)'] * 0.75 +
    grouped_filtered_combined_sentek['Sensor #2 (15 cm)'] * 0.15 +
    grouped_filtered_combined_sentek['Sensor #3 (25 cm)'] * 0.1
)

# Define a function to determine the time of day
def get_time_of_day(timestamp):
    hour = timestamp.hour
    if 6 <= hour < 10:
        return 'Morning'
    elif 10 <= hour < 19:
        return 'Daytime'
    else:
        return 'Night'

# Apply the function to create a 'Time of Day' column
grouped_filtered_combined_sentek['Time of Day'] = grouped_filtered_combined_sentek['Rounded Timestamp'].apply(get_time_of_day)

# Extract the date from the rounded timestamp
grouped_filtered_combined_sentek['Timestamp'] = grouped_filtered_combined_sentek['Rounded Timestamp'].dt.date

# Group by 'Date', 'Time of Day', and 'Team Number', then calculate the mean for sensor columns
grouped_by_time_of_day = grouped_filtered_combined_sentek.groupby(['Timestamp', 'Time of Day', 'Team Number']).agg({
    'Sensor #1 (5 cm)': 'mean',
    'Sensor #2 (15 cm)': 'mean',
    'Sensor #3 (25 cm)': 'mean',
    'Weighted VWC': 'mean'
}).reset_index()

# Display the resulting dataframes
grouped_filtered_combined_sentek


In [ ]:
time_of_day_weights = {
    'Morning': 0.25,
    'Daytime': 0.6,
    'Night': 0.15
}

# Calculate the weighted average of 'Weighted VWC' for each date
weighted_VWC = grouped_by_time_of_day.groupby(['Timestamp', 'Team Number']).apply(
    lambda x: (x['Weighted VWC'] * x['Time of Day'].map(time_of_day_weights)).sum() / x['Time of Day'].map(time_of_day_weights).sum()
).reset_index(name='Weighted Average VWC')

weighted_VWC['Team Number'] = weighted_VWC['Team Number'].str.extract('(\d+)').astype(int)
# Display the resulting dataframe
weighted_VWC


In [ ]:
texture_df = texture_df[texture_df["Sample Depth (in)"].isin(["0 - 12", "3 - 9", "4 - 9"])]
texture_df = texture_df.drop(columns=["Lat", "Lng"])


#Soil Water Capacity
#texture_df = pd.merge(texture_df, Team_Plot_ID, on='Plot ID', how='inner')
texture_df = texture_df.merge(Team_Plot_ID[['Plot ID', 'Team Number']], on='Plot ID', how='left')

texture_df

In [ ]:
merged_df = pd.merge(weighted_VWC, texture_df, on='Team Number', how='inner')

merged_df

In [ ]:
ET_mesonet['Timestamp'] = pd.to_datetime(ET_mesonet['Timestamp']).dt.date

merged_df = pd.merge(merged_df, ET_mesonet, on='Timestamp', how='inner')
merged_df

In [ ]:
import numpy as np
merged_df['PWP (%)'] = (((0.14) + (0.0033 * merged_df['Clay (%)'])) - (0.0022 * merged_df['Sand (%)']) + (0.0006 * merged_df['OMC (%) ']))*100
merged_df['Field Capacity (%)'] = ((0.31 + (0.0017 * merged_df['Clay (%)'])) - (0.0036 * merged_df['Sand (%)']) + (0.001 * merged_df['OMC (%) ']))*100
merged_df['Field Capacity (mm)'] = ((merged_df['Field Capacity (%)']/100)*135)
merged_df['Soil Water Storage (mm)'] = (merged_df['Weighted Average VWC']/100)*135
merged_df['Soil Water Depletion'] = (merged_df['Field Capacity (mm)'] - merged_df['Soil Water Storage (mm)'])
merged_df['Adjusted SWD'] = (merged_df['Soil Water Depletion'] + merged_df['Precipitation'])
merged_df['Crop Coefficient Corn (Kc)'] = (1.15)
merged_df['Evapotranspiration Corn (ET-Corn)'] = (merged_df['ETo Grass(mm)'] * merged_df['Crop Coefficient Corn (Kc)'])
merged_df['Total Water Needed based on Corn Kc'] = (merged_df['Evapotranspiration Corn (ET-Corn)'] + merged_df['Adjusted SWD'])
merged_df['Total Water Needed based on Corn Kc'] = np.where(merged_df['Total Water Needed based on Corn Kc'] < 0, 0, merged_df['Total Water Needed based on Corn Kc'])
merged_df['Recommended Irrigation'] = merged_df['Total Water Needed based on Corn Kc'] - merged_df['Precipitation']
merged_df['Recommended Irrigation'] = np.where(merged_df['Recommended Irrigation'] < 0, 0, merged_df['Recommended Irrigation'])


merged_df

In [ ]:
output_path = 'merged_df_TERMINADO.xlsx'
merged_df.to_excel(output_path, index=False)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import panel as pn

# Enable Panel extension
pn.extension(sizing_mode='stretch_width')

# Use the merged DataFrame 'merged_df'
irrigation_data = merged_df.copy()

# Convert 'Timestamp' to datetime if it's not already
irrigation_data['Timestamp'] = pd.to_datetime(irrigation_data['Timestamp'])

# Remove duplicate timestamps for the same irrigation recommendation from irrigation_data
irrigation_data = irrigation_data.sort_values('Timestamp').drop_duplicates(subset=['Timestamp'], keep='first')

# Create a list of unique team numbers
team_numbers = irrigation_data['Team Number'].unique()

# Create a dropdown menu for Team Numbers
team_selector = pn.widgets.Select(name='Select Team Number', options=team_numbers.tolist(), value=team_numbers[0])

# Create a date range slider for selecting timestamps
date_range_slider = pn.widgets.DateRangeSlider(name='Date Range', 
                                               start=irrigation_data['Timestamp'].min(), 
                                               end=irrigation_data['Timestamp'].max(), 
                                               value=(irrigation_data['Timestamp'].min(), irrigation_data['Timestamp'].max()))

# Function to update the graph based on the selected team and date range
def update_graph(selected_team, date_range):
    # Create an empty figure
    fig = go.Figure()

    # Filter data for the selected team and date range
    filtered_data = irrigation_data[
        (irrigation_data['Team Number'] == selected_team) &
        (irrigation_data['Timestamp'] >= date_range[0]) &
        (irrigation_data['Timestamp'] <= date_range[1])
    ]

    # Add traces for Precipitation and Recommended Irrigation
    fig.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=filtered_data['Precipitation'],
                             mode='lines', name='Precipitation'))
    fig.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=filtered_data['Recommended Irrigation'],
                             mode='lines', name='Recommended Irrigation'))

    # Update the layout for the entire chart
    fig.update_layout(
        title=f"Precipitation and Recommended Irrigation for Team {selected_team} Over Time",
        xaxis_title="Timestamp",
        yaxis_title="Water Depletion in mm",
        hovermode="x unified"
    )

    return fig

# Panel plot
@pn.depends(team_selector.param.value, date_range_slider.param.value)
def plot(selected_team, date_range):
    return pn.pane.Plotly(update_graph(selected_team, date_range))

# Layout the dashboard
dashboard_irrigation = pn.Column(
    pn.pane.Markdown("# Irrigation and Precipitation Dashboard", width=1400),
    team_selector,
    date_range_slider,
    plot,
    sizing_mode='fixed'
)



# Show the dashboard
dashboard_irrigation.servable()
